# Assignment 2: Tsetlin Machine for Breast Cancer Classification

This notebook implements and analyzes a Tsetlin Machine based on the concepts from Chapter 1 of "An Introduction to Tsetlin Machines". Using a small dataset of 6 patients, we will demonstrate:
1.  Manual Classification with pre-defined rules.
2.  Automatic Rule Learning using Type I and Type II feedback.
3.  The effect of the hyperparameters `forget_value` and `memorize_value` on the learned rules.

--- 

## Task 1: Manual Classification with Pre-defined Rules

First, we define the dataset for the 6 patients as described in the lecture. We then manually implement three rules (R1, R2, and R3) from the text.

* R1 and R2 are identical rules that vote for `Recurrence`.
* R3 is a rule that votes for `Non-Recurrence`.

Classification is performed using a majority vote among the rules. We calculate a vote sum where `Recurrence` rules add +1 and `Non-Recurrence` rules subtract -1. A positive final sum predicts `Recurrence`.

### Code: Data Setup and Manual Rules

In [2]:
import pandas as pd
from random import random, choice

# Define the dataset for the 6 patients as described in the lecture
patients = [
    {
        "lt40": False,
        "ge40": True,
        "premeno": False,
        "inv_0_2": False,
        "inv_3_5": True,
        "inv_6_8": False,
        "deg_malig_1": False,
        "deg_malig_2": False,
        "deg_malig_3": True,
        "recurrence": True,
    },
    {
        "lt40": True,
        "ge40": False,
        "premeno": False,
        "inv_0_2": True,
        "inv_3_5": False,
        "inv_6_8": False,
        "deg_malig_1": False,
        "deg_malig_2": False,
        "deg_malig_3": True,
        "recurrence": False,
    },
    {
        "lt40": False,
        "ge40": True,
        "premeno": False,
        "inv_0_2": False,
        "inv_3_5": False,
        "inv_6_8": True,
        "deg_malig_1": False,
        "deg_malig_2": False,
        "deg_malig_3": True,
        "recurrence": True,
    },
    {
        "lt40": False,
        "ge40": True,
        "premeno": False,
        "inv_0_2": True,
        "inv_3_5": False,
        "inv_6_8": False,
        "deg_malig_1": False,
        "deg_malig_2": True,
        "deg_malig_3": False,
        "recurrence": False,
    },
    {
        "lt40": False,
        "ge40": False,
        "premeno": True,
        "inv_0_2": True,
        "inv_3_5": False,
        "inv_6_8": False,
        "deg_malig_1": False,
        "deg_malig_2": False,
        "deg_malig_3": True,
        "recurrence": True,
    },
    {
        "lt40": False,
        "ge40": False,
        "premeno": True,
        "inv_0_2": True,
        "inv_3_5": False,
        "inv_6_8": False,
        "deg_malig_1": True,
        "deg_malig_2": False,
        "deg_malig_3": False,
        "recurrence": False,
    },
]

df = pd.DataFrame(patients)

In [3]:
# Define the three manual rules from the lecture
def R1(row):
    # Rule for Recurrence: Deg-malig 3 AND NOT Menopause lt40
    return row["deg_malig_3"] and (not row["lt40"])


def R2(row):
    # Identical rule for Recurrence to strengthen the vote
    return row["deg_malig_3"] and (not row["lt40"])


def R3(row):
    # Rule for Non-Recurrence: Inv-nodes 0-2
    return row["inv_0_2"]


# Apply the rules to each patient in the DataFrame
df["R1"] = df.apply(R1, axis=1)
df["R2"] = df.apply(R2, axis=1)
df["R3"] = df.apply(R3, axis=1)

# Calculate the vote sum and the final prediction
df["vote_sum"] = df["R1"].astype(int) + df["R2"].astype(int) - df["R3"].astype(int)
df["pred_task1"] = df["vote_sum"] > 0

# Display the final results
df[["R1", "R2", "R3", "vote_sum", "pred_task1", "recurrence"]]

,R1,R2,R3,vote_sum,pred_task1,recurrence
0,True,True,False,2,True,True
1,False,False,True,-1,False,False
2,True,True,False,2,True,True
3,False,False,True,-1,False,False
4,True,True,True,1,True,True
5,False,False,True,-1,False,False


Analysis of Task 1 Results:

The results above perfectly match those of the lecture. We can see that the combination of the three rules and the majority vote mechanism correctly classifies all 6 patients. Patient 5 is a prime example of the system's robustness, although rule R3 votes incorrectly for Non-Recurrence, the two correct votes from R1 and R2 win the majority, leading to the correct final prediction. This demonstrates how a Tsetlin Machine integrates rule-based and summation-based decision-making.

--- 
## Task 2 & 3: Learning Rules with Feedback

Now, we move to the automatic learning process. We implement a `Memory` class to track how strongly each literal (f.ex:`lt40` or `NOT lt40`) is memorized, on a scale from 1 (Forgotten) to 10 (Memorized). The learning is driven by two types of feedback:

**Type I Feedback (Recognize/Erase):** This feedback is used when a rule is exposed to an example from its own class. It reinforces patterns that match the example (Recognize) and weakens those that do not (Erase). The goal is to learn frequent patterns.

**Type II Feedback (Reject):** This feedback is used when a rule encounters an example from a different class. If the rule's condition is incorrectly met, this feedback makes the rule more specific to prevent it from making the same mistake again. The goal is to make the learned patterns discriminative.

We will now train rules for both `Recurrence` and `Non-Recurrence` using different `forget_value` and `memorize_value` settings to observe their impact. These values control how quickly a rule memorizes new information versus how quickly it forgets in the absence of observations.

### Code: Helper Functions and Memory Class

In [4]:
def evaluate_condition(observation, condition):
    truth_value_of_condition = True
    for feature in observation:
        if feature in condition and not observation[feature]:
            truth_value_of_condition = False
            break
        if "NOT " + feature in condition and observation[feature]:
            truth_value_of_condition = False
            break
    return truth_value_of_condition


class Memory:
    def __init__(self, forget_value, memorize_value, memory):
        self.memory = memory
        self.forget_value = forget_value
        self.memorize_value = memorize_value

    def get_memory(self):
        return self.memory

    def get_literals(self):
        return list(self.memory.keys())

    def get_condition(self):
        condition = []
        for literal in self.memory:
            if self.memory[literal] >= 6:
                condition.append(literal)
        return condition

    def memorize(self, literal):
        if random() <= self.memorize_value and self.memory[literal] < 10:
            self.memory[literal] += 1

    def forget(self, literal):
        if random() <= self.forget_value and self.memory[literal] > 1:
            self.memory[literal] -= 1

    def memorize_always(self, literal):
        if self.memory[literal] < 10:
            self.memory[literal] += 1


def type_i_feedback(observation, memory):
    remaining_literals = memory.get_literals()
    if evaluate_condition(observation, memory.get_condition()):
        for feature in observation:
            if feature == "recurrence":
                continue
            if observation[feature]:
                memory.memorize(feature)
                remaining_literals.remove(feature)
            elif not observation[feature]:
                memory.memorize("NOT " + feature)
                remaining_literals.remove("NOT " + feature)
    for literal in remaining_literals:
        memory.forget(literal)


def type_ii_feedback(observation, memory):
    if evaluate_condition(observation, memory.get_condition()):
        for feature in observation:
            if feature == "recurrence":
                continue
            if not observation[feature]:
                memory.memorize_always(feature)
            elif observation[feature]:
                memory.memorize_always("NOT " + feature)

### Analysis of Learned Rules

Below, we train rules and analyze how the `forget_value` and `memorize_value` hyperparameters affect the complexity of the learned patterns.

#### Case 1: High Forgetting (forget=0.8, memorize=0.2)
With a high `forget_value`, the machine will quickly forget literals that are not frequently observed. Only the strongest and most consistent patterns will be retained, leading to simpler, more general rules.

In [11]:
# -- Learn a rule for Recurrence --
cancer_rule_recurrence = Memory(
    forget_value=0.8,
    memorize_value=0.2,
    memory={
        "lt40": 5,
        "NOT lt40": 5,
        "ge40": 5,
        "NOT ge40": 5,
        "premeno": 5,
        "NOT premeno": 5,
        "inv_0_2": 5,
        "NOT inv_0_2": 5,
        "inv_3_5": 5,
        "NOT inv_3_5": 5,
        "inv_6_8": 5,
        "NOT inv_6_8": 5,
        "deg_malig_1": 5,
        "NOT deg_malig_1": 5,
        "deg_malig_2": 5,
        "NOT deg_malig_2": 5,
        "deg_malig_3": 5,
        "NOT deg_malig_3": 5,
    },
)

for i in range(5000):
    observation = choice(patients)
    if observation["recurrence"]:
        type_i_feedback(observation, cancer_rule_recurrence)
    else:
        type_ii_feedback(observation, cancer_rule_recurrence)

print("Recurrence Rule (F:0.8, M:0.2):")
print("IF " + " AND ".join(cancer_rule_recurrence.get_condition()) + " THEN Recurrence")

Recurrence Rule (F:0.8, M:0.2):
IF NOT lt40 AND NOT deg_malig_1 AND NOT deg_malig_2 AND deg_malig_3 THEN Recurrence


In [6]:
# -- Learn a rule for Non-Recurrence --
cancer_rule_non_recurrence = Memory(
    forget_value=0.8,
    memorize_value=0.2,
    memory={
        "lt40": 5,
        "NOT lt40": 5,
        "ge40": 5,
        "NOT ge40": 5,
        "premeno": 5,
        "NOT premeno": 5,
        "inv_0_2": 5,
        "NOT inv_0_2": 5,
        "inv_3_5": 5,
        "NOT inv_3_5": 5,
        "inv_6_8": 5,
        "NOT inv_6_8": 5,
        "deg_malig_1": 5,
        "NOT deg_malig_1": 5,
        "deg_malig_2": 5,
        "NOT deg_malig_2": 5,
        "deg_malig_3": 5,
        "NOT deg_malig_3": 5,
    },
)

for i in range(5000):
    observation = choice(patients)
    if not observation["recurrence"]:
        type_i_feedback(observation, cancer_rule_non_recurrence)
    else:
        type_ii_feedback(observation, cancer_rule_non_recurrence)

print("\nNon-Recurrence Rule (F:0.8, M:0.2):")
print("IF " + " AND ".join(cancer_rule_non_recurrence.get_condition()) + " THEN Non-Recurrence")


Non-Recurrence Rule (F:0.8, M:0.2):
IF inv_0_2 AND NOT inv_3_5 AND NOT inv_6_8 AND NOT deg_malig_3 THEN Non-Recurrence


The learned Recurrence rule is a logical equivalent of the high-frequency pattern: `if Deg-malig 3 and not lt40 then Recurrence`. The high forget rate forces the machine to discard noise and keep only the most essential features.

#### Case 2: Balanced Memorization and Forgetting (forget=0.5, memorize=0.5)
Here, a balance is struck between remembering and forgetting. The machine can now learn "mid-frequency" patterns that may be present in *some*, but not all, examples of a class.

In [7]:
# -- Learn a rule for Recurrence --
cancer_rule_recurrence = Memory(
    forget_value=0.5,
    memorize_value=0.5,
    memory={
        "lt40": 5,
        "NOT lt40": 5,
        "ge40": 5,
        "NOT ge40": 5,
        "premeno": 5,
        "NOT premeno": 5,
        "inv_0_2": 5,
        "NOT inv_0_2": 5,
        "inv_3_5": 5,
        "NOT inv_3_5": 5,
        "inv_6_8": 5,
        "NOT inv_6_8": 5,
        "deg_malig_1": 5,
        "NOT deg_malig_1": 5,
        "deg_malig_2": 5,
        "NOT deg_malig_2": 5,
        "deg_malig_3": 5,
        "NOT deg_malig_3": 5,
    },
)

for i in range(5000):
    observation = choice(patients)
    if observation["recurrence"]:
        type_i_feedback(observation, cancer_rule_recurrence)
    else:
        type_ii_feedback(observation, cancer_rule_recurrence)

print("Recurrence Rule (F:0.5, M:0.5):")
print("IF " + " AND ".join(cancer_rule_recurrence.get_condition()) + " THEN Recurrence")

Recurrence Rule (F:0.5, M:0.5):
IF NOT lt40 AND ge40 AND NOT premeno AND NOT inv_0_2 AND NOT deg_malig_1 AND NOT deg_malig_2 AND deg_malig_3 THEN Recurrence


In [8]:
# -- Learn a rule for Non-Recurrence --
cancer_rule_non_recurrence = Memory(
    forget_value=0.5,
    memorize_value=0.5,
    memory={
        "lt40": 5,
        "NOT lt40": 5,
        "ge40": 5,
        "NOT ge40": 5,
        "premeno": 5,
        "NOT premeno": 5,
        "inv_0_2": 5,
        "NOT inv_0_2": 5,
        "inv_3_5": 5,
        "NOT inv_3_5": 5,
        "inv_6_8": 5,
        "NOT inv_6_8": 5,
        "deg_malig_1": 5,
        "NOT deg_malig_1": 5,
        "deg_malig_2": 5,
        "NOT deg_malig_2": 5,
        "deg_malig_3": 5,
        "NOT deg_malig_3": 5,
    },
)

for i in range(5000):
    observation = choice(patients)
    if not observation["recurrence"]:
        type_i_feedback(observation, cancer_rule_non_recurrence)
    else:
        type_ii_feedback(observation, cancer_rule_non_recurrence)

print("\nNon-Recurrence Rule (F:0.5, M:0.5):")
print("IF " + " AND ".join(cancer_rule_non_recurrence.get_condition()) + " THEN Non-Recurrence")


Non-Recurrence Rule (F:0.5, M:0.5):
IF NOT premeno AND inv_0_2 AND NOT inv_3_5 AND NOT inv_6_8 AND NOT deg_malig_1 THEN Non-Recurrence


The rules are noticeably more complex. The Recurrence rule now includes `ge40`. This represents a pattern that is true for two of the three `Recurrence` patients, which the balanced parameters allow the machine to learn and retain.

#### Case 3: High Memorization (forget=0.2, memorize=0.8)
With a high `memorize_value`, the machine will very easily strengthen its memory of observed literals. This leads to highly specific and detailed rules tailored to the examples in the training set.

In [9]:
# -- Learn a rule for Recurrence --
cancer_rule_recurrence = Memory(
    forget_value=0.2,
    memorize_value=0.8,
    memory={
        "lt40": 5,
        "NOT lt40": 5,
        "ge40": 5,
        "NOT ge40": 5,
        "premeno": 5,
        "NOT premeno": 5,
        "inv_0_2": 5,
        "NOT inv_0_2": 5,
        "inv_3_5": 5,
        "NOT inv_3_5": 5,
        "inv_6_8": 5,
        "NOT inv_6_8": 5,
        "deg_malig_1": 5,
        "NOT deg_malig_1": 5,
        "deg_malig_2": 5,
        "NOT deg_malig_2": 5,
        "deg_malig_3": 5,
        "NOT deg_malig_3": 5,
    },
)

for i in range(5000):
    observation = choice(patients)
    if observation["recurrence"]:
        type_i_feedback(observation, cancer_rule_recurrence)
    else:
        type_ii_feedback(observation, cancer_rule_recurrence)

print("Recurrence Rule (F:0.2, M:0.8):")
print("IF " + " AND ".join(cancer_rule_recurrence.get_condition()) + " THEN Recurrence")

Recurrence Rule (F:0.2, M:0.8):
IF NOT lt40 AND ge40 AND NOT premeno AND NOT inv_0_2 AND NOT inv_3_5 AND inv_6_8 AND NOT deg_malig_1 AND NOT deg_malig_2 AND deg_malig_3 THEN Recurrence


In [10]:
# -- Learn a rule for Non-Recurrence --
cancer_rule_non_recurrence = Memory(
    forget_value=0.2,
    memorize_value=0.8,
    memory={
        "lt40": 5,
        "NOT lt40": 5,
        "ge40": 5,
        "NOT ge40": 5,
        "premeno": 5,
        "NOT premeno": 5,
        "inv_0_2": 5,
        "NOT inv_0_2": 5,
        "inv_3_5": 5,
        "NOT inv_3_5": 5,
        "inv_6_8": 5,
        "NOT inv_6_8": 5,
        "deg_malig_1": 5,
        "NOT deg_malig_1": 5,
        "deg_malig_2": 5,
        "NOT deg_malig_2": 5,
        "deg_malig_3": 5,
        "NOT deg_malig_3": 5,
    },
)

for i in range(5000):
    observation = choice(patients)
    if not observation["recurrence"]:
        type_i_feedback(observation, cancer_rule_non_recurrence)
    else:
        type_ii_feedback(observation, cancer_rule_non_recurrence)

print("\nNon-Recurrence Rule (F:0.2, M:0.8):")
print("IF " + " AND ".join(cancer_rule_non_recurrence.get_condition()) + " THEN Non-Recurrence")


Non-Recurrence Rule (F:0.2, M:0.8):
IF NOT lt40 AND NOT ge40 AND premeno AND inv_0_2 AND NOT inv_3_5 AND NOT inv_6_8 AND deg_malig_1 AND NOT deg_malig_2 AND NOT deg_malig_3 THEN Non-Recurrence


As expected, these rules are the most complex. They include many literals and are highly specific. With a tiny dataset like this, this behavior is likely to overfitting, where the rule memorizes individual examples rather than learning a general pattern.

--- 
## Conclusion

This assignment successfully demonstrated the core concepts of a Tsetlin Machine:

1.  Task 1 showed how multiple rules cooperate via majority voting to produce a robust classification.
2.  Task 2 and 3 showed the dynamic learning process, where rules are shaped by Type I and Type II feedback.
3.  We observed a clear relationship between the hyperparameters (`forget_value`, `memorize_value`) and the complexity of the learned rules. This is a fundamental trade-off, allowing a user to control whether the machine learns general, simple patterns or specific, complex ones.